In [ ]:
from icecube import icetray, dataclasses, dataio
import numpy as np
import glob
import json

files = glob.glob("./search_data/*/Alerts*.i3")
files.sort()
print(files)


In [ ]:
event_dict = {}
for file in files:
    ofile = dataio.I3File(file)
    while ofile.more():
        phys = ofile.pop_physics()
        if phys.Has("Alert_JSON"):
            header = phys['I3EventHeader']
            #print(header.start_time.mod_julian_day_double)
            gfu_alert = json.loads(phys['Alert_JSON'].value)
            select = 'None'
            if gfu_alert['pass_tight']:
                select = 'Gold'
            else:
                if gfu_alert['pass_loose']:
                    select = 'Bronze'
            #print(file)
            print(f"Run: {gfu_alert['run_id']:6} Event: {gfu_alert['event_id']:8} Subevent:{header.sub_event_id:3} MJD: {header.start_time.mod_julian_day_double:0.2f} ",\
                  f"RA: {np.rad2deg(gfu_alert['ra']):0.2f} DEC: {np.rad2deg(gfu_alert['dec']):0.2f} ERR50: {np.rad2deg(gfu_alert['err_50']):0.2f} ERR90: {np.rad2deg(gfu_alert['err_90']):0.2f} ",\
                  f"E_muon:{(gfu_alert['E_muon']/1000.0):0.2f}  E_nu:{(gfu_alert['E_nu_peak']/1000.0):0.2f} Sgns: {gfu_alert['signalness']:0.2f}",\
                  f"Selection: {select:10}")
            key = str(gfu_alert['run_id']) + '_' + str(gfu_alert['event_id'])
            print(key)                  
            event_dict[key] = {
                'MJD' : header.start_time.mod_julian_day_double,
                'RA' : np.rad2deg(gfu_alert['ra']),
                'DEC' : np.rad2deg(gfu_alert['dec']),
                'ERR50' : np.rad2deg(gfu_alert['err_50']),
                'ERR90' : np.rad2deg(gfu_alert['err_90']),
                'E_NU' : gfu_alert['E_nu_peak'],
                'SGNS' : gfu_alert['signalness'],
                'FAR' : gfu_alert['yearly_rate'],
                'SELECTION' : select
            }
with open('gfu_events.txt', 'w') as outfile:
    json.dump(event_dict, outfile)


            
            
            
#        fm_hese = phys['FilterMask']['HESEFilter_15']
#        if fm_hese.condition_passed:
#            if phys.Has('HESE_CausalQTot'):
#                cqtot = phys['HESE_CausalQTot'].value
#                print('HESE cqtot:',cqtot)